In [1]:
import pandas as pd
import logging

from src import eda, datasets

In [2]:
logging.getLogger().setLevel(logging.DEBUG)

In [3]:
project_pack = datasets.DatasetPack(restore=True)

INFO:root:Restoring backup: /Users/bubblegum_doubledrops/Library/Mobile Documents/com~apple~CloudDocs/0prio - Important heavy backups/IronHack/big_projects/end-to-end-data-analysis/datasets/_dataset_pack_end-to-end-data-analysis.json
INFO:root:Restoring dataframe from backup: /Users/bubblegum_doubledrops/Library/Mobile Documents/com~apple~CloudDocs/0prio - Important heavy backups/IronHack/big_projects/end-to-end-data-analysis/datasets/df_bandcamp_sales.pkl
INFO:root:Restoring dataframe from backup: /Users/bubblegum_doubledrops/Library/Mobile Documents/com~apple~CloudDocs/0prio - Important heavy backups/IronHack/big_projects/end-to-end-data-analysis/datasets/df_discogs_artists.pkl
INFO:root:Restoring dataframe from backup: /Users/bubblegum_doubledrops/Library/Mobile Documents/com~apple~CloudDocs/0prio - Important heavy backups/IronHack/big_projects/end-to-end-data-analysis/datasets/df_discogs_releases.pkl
INFO:root:Backup restored successfully from /Users/bubblegum_doubledrops/Library/M

In [4]:
for label, dataset in project_pack.dictionary.items():
    print(f"================= {label} =================")
    dataset.dataframe.info()
    print()

================= df_bandcamp_sales =================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   _id               1000000 non-null  object 
 1   item_type         1000000 non-null  object 
 2   utc_date          1000000 non-null  float64
 3   country_code      1000000 non-null  object 
 4   country           1000000 non-null  object 
 5   slug_type         988416 non-null   object 
 6   item_description  999977 non-null   object 
 7   art_id            764520 non-null   float64
 8   url               1000000 non-null  object 
 9   releases          11584 non-null    float64
 10  artist_name       999990 non-null   object 
 11  album_title       356461 non-null   object 
 12  amount_paid_usd   1000000 non-null  float64
 13  amount_over_fmt   119133 non-null   object 
 14  item_slug         21742 non-null    object 
d

In [ ]:
# ONLY RUN ONCE per session!!!
auto_eda_pack = {label: eda.Auto_EDA(dataset) for label, dataset in project_pack.dictionary.items()}

In [6]:
for label, eda in auto_eda_pack.items():
    logging.info(f"================= {label} =================")
    logging.info(f"Number of rows before cleanup: {eda.init_rows}")
    eda.auto_cleanup()
    logging.info(f"Number of rows after cleanup: {eda.count_rows()}")

INFO:root:================= df_bandcamp_sales =================
INFO:root:Number of rows before cleanup: 1000000
INFO:root:df_bandcamp_sales has no empty spaces.
INFO:root:df_bandcamp_sales has no duplicates.
INFO:root:Number of rows after cleanup: 1000000
INFO:root:================= df_discogs_artists =================
INFO:root:Number of rows before cleanup: 9194907
INFO:root:df_discogs_artists has no empty spaces.
INFO:root:df_discogs_artists has no duplicates.
INFO:root:Number of rows after cleanup: 9194907
INFO:root:================= df_discogs_releases =================
INFO:root:Number of rows before cleanup: 17372035
INFO:root:df_discogs_releases has no empty spaces.
INFO:root:df_discogs_releases had duplicates. They had been dropped and index was reset.
INFO:root:Number of rows after cleanup: 17246710


In [7]:
# this code varifies that the cleanup took place in the original data frame restored from pickle:
project_pack.dictionary['df_discogs_releases'].dataframe.info()
# before: 17372035 rows
# after cleanup: 17246710 rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17246710 entries, 0 to 17246709
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   release_id  int64  
 1   country     object 
 2   year        float64
 3   genre       object 
 4   format      object 
dtypes: float64(1), int64(1), object(3)
memory usage: 657.9+ MB


In [8]:
# now I want to backup the changed data frame:
project_pack.dictionary['df_discogs_releases'].backup()

INFO:root:Backup file is created: /Users/bubblegum_doubledrops/Library/Mobile Documents/com~apple~CloudDocs/0prio - Important heavy backups/IronHack/big_projects/end-to-end-data-analysis/datasets/df_discogs_releases.pkl


In [9]:
for label, eda in auto_eda_pack.items():
    print(f"================= {label} =================")
    df_nulls = eda.count_nulls()
    print(df_nulls)
    print()

================= df_bandcamp_sales =================
                   is_na   not_na  na_percent na_percent_pretty
releases          988416    11584     98.8416            98.84%
item_slug         978258    21742     97.8258            97.83%
amount_over_fmt   880867   119133     88.0867            88.09%
album_title       643539   356461     64.3539            64.35%
art_id            235480   764520     23.5480            23.55%
slug_type          11584   988416      1.1584             1.16%
item_description      23   999977      0.0023             0.00%
artist_name           10   999990      0.0010             0.00%
_id                    0  1000000      0.0000             0.00%
item_type              0  1000000      0.0000             0.00%
utc_date               0  1000000      0.0000             0.00%
country_code           0  1000000      0.0000             0.00%
country                0  1000000      0.0000             0.00%
url                    0  1000000      0.0000     

In [11]:
# using the information from dataframes from the previous output,
# dropping columns with all null values

# manually:
# project_pack.dictionary['df_discogs_releases'].dataframe.drop(columns= ["copy paste from the data frame print"], inplace=True) 

treshold = 70

# automatically:
for label, eda in auto_eda_pack.items():
    logging.info(f"================= {label} =================")
    logging.info(f"Number of columns before cleanup: {eda.init_cols}")

    # TODO: stop calling the same expensive method over and over
    df_nulls = eda.count_nulls()

    columns_to_drop = df_nulls[df_nulls['na_percent'] > treshold].index.to_list()
    logging.info(f'Dropping {len(columns_to_drop)} empty columns: {columns_to_drop}')
    eda.df.drop(columns=columns_to_drop, inplace=True)
    logging.info(f"Number of columns after cleanup: {eda.count_cols()}")

    logging.info(f'Backing up result dataframe')
    eda.dataset.backup
    logging.info("\n")


INFO:root:================= df_bandcamp_sales =================
INFO:root:Number of columns before cleanup: 15
INFO:root:Dropping 0 empty columns: []
INFO:root:Number of columns after cleanup: 12
INFO:root:Backing up result dataframe
INFO:root:

INFO:root:================= df_discogs_artists =================
INFO:root:Number of columns before cleanup: 12
INFO:root:Dropping 0 empty columns: []
INFO:root:Number of columns after cleanup: 4
INFO:root:Backing up result dataframe
INFO:root:

INFO:root:================= df_discogs_releases =================
INFO:root:Number of columns before cleanup: 5
INFO:root:Dropping 0 empty columns: []
INFO:root:Number of columns after cleanup: 5
INFO:root:Backing up result dataframe
INFO:root:



In [12]:
for label, eda in auto_eda_pack.items():
    print(f"================= {label} =================")
    df_nulls = eda.count_nulls()
    print(df_nulls)
    print()

================= df_bandcamp_sales =================
                   is_na   not_na  na_percent na_percent_pretty
album_title       643539   356461     64.3539            64.35%
art_id            235480   764520     23.5480            23.55%
slug_type          11584   988416      1.1584             1.16%
item_description      23   999977      0.0023             0.00%
artist_name           10   999990      0.0010             0.00%
_id                    0  1000000      0.0000             0.00%
item_type              0  1000000      0.0000             0.00%
utc_date               0  1000000      0.0000             0.00%
country_code           0  1000000      0.0000             0.00%
country                0  1000000      0.0000             0.00%
url                    0  1000000      0.0000             0.00%
amount_paid_usd        0  1000000      0.0000             0.00%

================= df_discogs_artists =================
                     is_na   not_na  na_percent na_percent

In [13]:
%%capture output
for label, eda in auto_eda_pack.items():
    eda.print_eda_report()

In [14]:
print(output.stdout)

report_path = f'{project_pack.get_project_root()}/reports/eda_raw_report.md'

with open(report_path, "w") as f:
    f.write(output.stdout)

print(f"Output has been saved to file:\n{report_path}")

================= df_bandcamp_sales =================
df_bandcamp_sales has shape (1000000, 12)

df_bandcamp_sales has numerical data in columns: ['utc_date', 'art_id', 'amount_paid_usd']
- Column "utc_date" has 999990 unique values.
- Column "art_id" has 271430 unique values.
- Column "amount_paid_usd" has 5866 unique values.

df_bandcamp_sales has categorical data in columns: ['_id', 'item_type', 'country_code', 'country', 'slug_type', 'item_description', 'url', 'artist_name', 'album_title']
- Column "_id" has 1000000 unique values.
- Column "item_type" has 4 unique values.
  -- Unique values are:
 ['a' 'p' 't' 'b']
- Column "country_code" has 186 unique values.
- Column "country" has 186 unique values.
- Column "slug_type" has 3 unique values.
  -- Unique values are:
 ['a' 't' 'p' nan]
- Column "item_description" has 336985 unique values.
- Column "url" has 374473 unique values.
- Column "artist_name" has 159746 unique values.
- Column "album_title" has 103697 unique values.

======

### From here onwards:
Due to the project deadline src package modules are used in a limited way and are subject to refactoring somewhen in future.

DF labels for reference:
- df_bandcamp_sales
- df_discogs_artists
- df_discogs_releases

In [13]:
print("Copy the output as code to run in the next cell")
# TODO: implement DatasetPack.labels() to return a list
for label in project_pack.dictionary.keys():
    print(f"{label} = project_pack.dictionary['{label}'].dataframe")

Copy the output as code to run in the next cell
df_bandcamp_sales = project_pack.dictionary['df_bandcamp_sales'].dataframe
df_discogs_artists = project_pack.dictionary['df_discogs_artists'].dataframe
df_discogs_releases = project_pack.dictionary['df_discogs_releases'].dataframe


In [14]:
df_bandcamp_sales = project_pack.dictionary['df_bandcamp_sales'].dataframe
df_discogs_artists = project_pack.dictionary['df_discogs_artists'].dataframe
df_discogs_releases = project_pack.dictionary['df_discogs_releases'].dataframe

In [15]:
print("Copy the output as code to run in the next cell")
for eda in auto_eda_pack:
    print(f"num_cols_{eda.dataset.label} = {eda.get_numerical_columns()}")

Copy the output as code to run in the next cell
num_cols_df_bandcamp_sales = ['utc_date', 'art_id', 'amount_paid_usd']
num_cols_df_discogs_artists = ['Unnamed: 0', 'artist_id']
num_cols_df_discogs_releases = ['release_id', 'year']


In [17]:
num_cols_df_bandcamp_sales = ['utc_date', 'art_id', 'amount_paid_usd']
num_cols_df_discogs_artists = ['Unnamed: 0', 'artist_id']
num_cols_df_discogs_releases = ['release_id', 'year']

In [18]:
print("Copy the output as code to run in the next separate cells,\n each for one line")
for eda in auto_eda_pack:
    print(f"{eda.dataset.label}[num_cols_{eda.dataset.label}]")
    print(f"{eda.dataset.label}[num_cols_{eda.dataset.label}].describe().T")

Copy the output as code to run in the next separate cells,
 each for one line
df_bandcamp_sales[num_cols_df_bandcamp_sales]
df_bandcamp_sales[num_cols_df_bandcamp_sales].describe().T
df_discogs_artists[num_cols_df_discogs_artists]
df_discogs_artists[num_cols_df_discogs_artists].describe().T
df_discogs_releases[num_cols_df_discogs_releases]
df_discogs_releases[num_cols_df_discogs_releases].describe().T


In [19]:
df_bandcamp_sales[num_cols_df_bandcamp_sales]

,utc_date,art_id,amount_paid_usd
0,1.599689e+09,2.064053e+08,9.99
1,1.599689e+09,2.984242e+09,1.30
2,1.599689e+09,3.320495e+09,3.90
3,1.599689e+09,NaN,12.39
4,1.599689e+09,3.428873e+09,1.00
...,...,...,...
999995,1.601647e+09,NaN,20.00
999996,1.601647e+09,3.708391e+09,20.20
999997,1.601647e+09,3.580857e+08,1.00
999998,1.601647e+09,NaN,6.00


In [20]:
df_bandcamp_sales[num_cols_df_bandcamp_sales].describe().T

,count,mean,std,min,25%,50%,75%,max
utc_date,1000000.0,1.600750e+09,6.049259e+05,1.599689e+09,1.600237e+09,1.600778e+09,1.601288e+09,1.601647e+09
art_id,764520.0,2.141628e+09,1.241930e+09,2.018000e+04,1.057849e+09,2.137613e+09,3.228946e+09,4.294959e+09
amount_paid_usd,1000000.0,8.931315e+00,1.249008e+01,2.000000e-01,2.000000e+00,6.000000e+00,1.163000e+01,1.286260e+03


In [21]:
df_discogs_artists[num_cols_df_discogs_artists]

,Unnamed: 0,artist_id
0,0,1
1,1,2
2,2,3
3,3,4
4,4,5
...,...,...
9194902,9194902,1275492
9194903,9194903,1275493
9194904,9194904,1275494
9194905,9194905,1275498


In [22]:
df_discogs_releases[num_cols_df_discogs_releases]

,release_id,year
0,1,1999.0
1,2,1998.0
2,3,1999.0
3,4,1999.0
4,5,1995.0
...,...,...
17246705,15731745,1980.0
17246706,15731748,1975.0
17246707,15731752,2020.0
17246708,15731758,2007.0


In [23]:
df_discogs_releases[num_cols_df_discogs_releases].describe().T

,count,mean,std,min,25%,50%,75%,max
release_id,17246710.0,7.828094e+06,4.545709e+06,1.0,3810774.0,7729440.5,11836531.5,15731760.0
year,15086359.0,1.995260e+03,1.786837e+01,1860.0,1983.0,1998.0,2010.0,2021.0


In [26]:
print("Copy the output as code to run in the next cell")
for eda in auto_eda_pack:
    print(f"cat_cols_{eda.dataset.label} = {eda.get_categorical_columns()}")

Copy the output as code to run in the next cell
cat_cols_df_bandcamp_sales = ['_id', 'item_type', 'country_code', 'country', 'slug_type', 'item_description', 'url', 'artist_name', 'currency', 'album_title']
cat_cols_df_discogs_artists = ['artist_data_quality', 'artist_name']
cat_cols_df_discogs_releases = ['country', 'genre', 'format']


In [27]:
cat_cols_df_bandcamp_sales = ['_id', 'item_type', 'country_code', 'country', 'slug_type', 'item_description', 'url', 'artist_name', 'currency', 'album_title']
cat_cols_df_discogs_artists = ['artist_data_quality', 'artist_name']
cat_cols_df_discogs_releases = ['country', 'genre', 'format']

In [28]:
print("Copy the output as code to run in the next separate cells,\n each for one line")
for eda in auto_eda_pack:
    print(f"{eda.dataset.label}[cat_cols_{eda.dataset.label}]")
    print(f"{eda.dataset.label}[cat_cols_{eda.dataset.label}].describe().T")

Copy the output as code to run in the next separate cells,
 each for one line
df_bandcamp_sales[cat_cols_df_bandcamp_sales]
df_bandcamp_sales[cat_cols_df_bandcamp_sales].describe().T
df_discogs_artists[cat_cols_df_discogs_artists]
df_discogs_artists[cat_cols_df_discogs_artists].describe().T
df_discogs_releases[cat_cols_df_discogs_releases]
df_discogs_releases[cat_cols_df_discogs_releases].describe().T


In [29]:
df_bandcamp_sales[cat_cols_df_bandcamp_sales]

,_id,item_type,country_code,country,slug_type,item_description,url,artist_name,currency,album_title
0,1599688803.5175&//girlbanddublin.bandcamp.com/...,a,gb,United Kingdom,a,Live at Vicar Street,//girlbanddublin.bandcamp.com/album/live-at-vi...,Girl Band,USD,NaN
1,1599688805.27838&//maharettarecords.bandcamp.c...,a,fi,Finland,a,Neurogen,//maharettarecords.bandcamp.com/album/neurogen,Jirah,GBP,NaN
2,1599688805.90646&//maharettarecords.bandcamp.c...,a,fi,Finland,a,The Last Snare Bender,//maharettarecords.bandcamp.com/album/the-last...,D-Ther,GBP,NaN
3,1599688806.94234&//alicesitski.bandcamp.com/al...,p,gb,United Kingdom,a,Limited Edition Compact Disc,//alicesitski.bandcamp.com/album/white-noise-tv,WHITE NOISE TV,EUR,WHITE NOISE TV
4,1599688809.07942&//linguaignota.bandcamp.com/t...,t,us,United States,t,O Ruthless Great Divine Director,//linguaignota.bandcamp.com/track/o-ruthless-g...,LINGUA IGNOTA,USD,NaN
...,...,...,...,...,...,...,...,...,...,...
999995,1601647409.59517&//eternalchampion.bandcamp.co...,p,ca,Canada,a,Black vinyl,//eternalchampion.bandcamp.com/album/ravening-...,Eternal Champion,USD,RAVENING IRON
999996,1601647409.61911&//goodmusic2020.bandcamp.com/...,a,us,United States,a,Good Music to Avert the Collapse of American D...,//goodmusic2020.bandcamp.com/album/good-music-...,Various Artists Working to Avert the Collapse ...,USD,NaN
999997,1601647409.63413&//geometriclullaby.bandcamp.c...,a,au,Australia,a,O'discordia,//geometriclullaby.bandcamp.com/album/odiscordia,w i n t e r q u i l t,USD,NaN
999998,1601647409.77243&//bewitcher.bandcamp.com/merc...,p,us,United States,p,Bewitcher Logo Patch,//bewitcher.bandcamp.com/merch/bewitcher-logo-...,Bewitcher,USD,NaN


In [30]:
df_bandcamp_sales[cat_cols_df_bandcamp_sales].describe().T

,count,unique,top,freq
_id,1000000,1000000,1599688803.5175&//girlbanddublin.bandcamp.com/...,1
item_type,1000000,4,a,481584
country_code,1000000,186,us,398999
country,1000000,186,United States,398999
slug_type,988416,3,a,669409
item_description,999977,336985,Limited Edition Cassette,6052
url,1000000,374473,//goodmusic2020.bandcamp.com/album/good-music-...,3479
artist_name,999990,159746,Various Artists,10931
currency,1000000,18,USD,455569
album_title,356461,103697,UNTITLED (Rise),1669


In [31]:
df_discogs_artists[cat_cols_df_discogs_artists]

,artist_data_quality,artist_name
0,Needs Vote,The Persuader
1,Correct,Mr. James Barth & A.D.
2,Correct,Josh Wink
3,Needs Vote,Johannes Heil
4,Needs Vote,Heiko Laux
...,...,...
9194902,Needs Major Changes,Antimatter (5)
9194903,Needs Vote,Matthew Sigley
9194904,Needs Vote,Krzysztof Fijalkowski
9194905,Needs Vote,Melinda Irtl


In [32]:
df_discogs_artists[cat_cols_df_discogs_artists].describe().T

,count,unique,top,freq
artist_data_quality,9194907,6,Needs Major Changes,6014661
artist_name,9194905,9194896,The Love Club (Detroit),2


In [34]:
df_discogs_releases[cat_cols_df_discogs_releases]

,country,genre,format
0,Sweden,Electronic,Vinyl
1,Sweden,Electronic,Vinyl
2,US,Electronic,CD
3,US,Electronic,CD
4,Germany,Electronic,CD
...,...,...,...
17246705,Peru,Pop,Vinyl
17246706,Mexico,Latin,Vinyl
17246707,Czech Republic,Electronic,Vinyl
17246708,Canada,Electronic,Vinyl


In [35]:
df_discogs_releases[cat_cols_df_discogs_releases].describe().T

,count,unique,top,freq
country,16796735,281,US,4013714
genre,17246707,15,Rock,4414291
format,17246710,57,Vinyl,7737704


## Backup

In [15]:
for dataset in project_pack.dictionary.values():
    dataset.backup()

project_pack.backup_pack()

INFO:root:Backup file is created: /Users/bubblegum_doubledrops/Library/Mobile Documents/com~apple~CloudDocs/0prio - Important heavy backups/IronHack/big_projects/end-to-end-data-analysis/datasets/df_bandcamp_sales.pkl
INFO:root:Backup file is created: /Users/bubblegum_doubledrops/Library/Mobile Documents/com~apple~CloudDocs/0prio - Important heavy backups/IronHack/big_projects/end-to-end-data-analysis/datasets/df_discogs_artists.pkl
INFO:root:Backup file is created: /Users/bubblegum_doubledrops/Library/Mobile Documents/com~apple~CloudDocs/0prio - Important heavy backups/IronHack/big_projects/end-to-end-data-analysis/datasets/df_discogs_releases.pkl
INFO:root:Backup not changed, skipping write: /Users/bubblegum_doubledrops/Library/Mobile Documents/com~apple~CloudDocs/0prio - Important heavy backups/IronHack/big_projects/end-to-end-data-analysis/datasets/_dataset_pack_end-to-end-data-analysis.json
